In [2]:
import random
    
QUERY_POOL = list(range(11))
random.shuffle(QUERY_POOL)
for index in range(4):
    # get the last 'batch_size_test' nodes as query nodes
    query_indices = QUERY_POOL[:3]
    QUERY_POOL = QUERY_POOL[3:]
    print(index,'query_indices', query_indices)

0 query_indices [7, 2, 8]
1 query_indices [3, 6, 1]
2 query_indices [4, 0, 9]
3 query_indices [5, 10]


In [4]:
import pandas as pd
# baic transformer Decoder model
import torch
import torch.nn as nn
import torch.nn.functional as Fun
import torch.optim as optim
import numpy as np
import xformers.ops as xops
import math 
from typing import Optional, Union
from torch import Tensor
import random

main_df = pd.read_csv('../data/RAW_data/adult.csv')
main_df.head()
DEVICE = 'cuda'
# DEVICE = 'cpu'
torch.random.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)

In [6]:
from ..data.preprocess import POOL_preprocess
X_trans, inference_package , _  = POOL_preprocess(main_df)
X_trans

ImportError: attempted relative import with no known parent package

## 5 FOLD (transformed input)

In [3]:
import numpy as np
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score
# import labelencoder
from sklearn.preprocessing import LabelEncoder

kf = KFold(n_splits=5, shuffle=True)
AUCS = []
main_df = pd.read_csv('adult.csv')
main_df =X_trans

# 進行5-fold交叉驗證
for index, (train_index, test_index) in enumerate(kf.split(main_df)):
    X_train, X_test = main_df.loc[train_index], main_df.loc[test_index]
    
    le = LabelEncoder()
    Y_train, Y_test = X_train['income'] , X_test['income']
    Y_train = le.fit_transform(Y_train)
    Y_test = le.transform(Y_test)
    
    X_train, X_test = X_train.drop(columns='income'), X_test.drop(columns='income')
    
    CAT = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
    le2 = LabelEncoder()
    le2.fit(main_df[CAT].astype(str).values.flatten())
    for column in CAT:
        le = LabelEncoder()
        X_train[column] = le2.transform(X_train[column].astype(str))
        X_test[column] = le2.transform(X_test[column].astype(str))
    
    print(f'Fold {index}:')
    print(f'X_train: {X_train.shape}, X_test: {X_test.shape}')
    print(f'Y_train: {Y_train.shape}, Y_test: {Y_test.shape}')
    
    model = xgb.XGBClassifier(random_state=42, n_jobs=-1)
    model.fit(X_train, Y_train)
    Y_prob = model.predict_proba(X_test)  # 预测概率
    auc = roc_auc_score(Y_test, Y_prob[:, 1])
    AUCS.append(auc)
    print(f'auc: {auc}')
print(f'Average AUC: {np.mean(AUCS)}')

Fold 0:
X_train: (39073, 14), X_test: (9769, 14)
Y_train: (39073,), Y_test: (9769,)
auc: 0.9215133961701206
Fold 1:
X_train: (39073, 14), X_test: (9769, 14)
Y_train: (39073,), Y_test: (9769,)
auc: 0.9221211330968312
Fold 2:
X_train: (39074, 14), X_test: (9768, 14)
Y_train: (39074,), Y_test: (9768,)
auc: 0.9205817394690794
Fold 3:
X_train: (39074, 14), X_test: (9768, 14)
Y_train: (39074,), Y_test: (9768,)
auc: 0.9171000039597399
Fold 4:
X_train: (39074, 14), X_test: (9768, 14)
Y_train: (39074,), Y_test: (9768,)
auc: 0.9190697360365623
Average AUC: 0.9200772017464665


## 5-FOLD (original input)

In [4]:
kf = KFold(n_splits=5, shuffle=True)
AUCS = []
# 進行5-fold交叉驗證
for index, (train_index, test_index) in enumerate(kf.split(main)):
    X_train, X_test = X_trans.loc[train_index], X_trans.loc[test_index]
    Y_train, Y_test = X_train['income'] - X_train['income'].min(), X_test['income'] - X_test['income'].min()
    X_train, X_test = X_train.drop(columns='income'), X_test.drop(columns='income')
    print(f'Fold {index}:')
    print(f'X_train: {X_train.shape}, X_test: {X_test.shape}')
    print(f'Y_train: {Y_train.shape}, Y_test: {Y_test.shape}')
    
    model = xgb.XGBClassifier()
    model.fit(X_train, Y_train)
    Y_prob = model.predict_proba(X_test)  # 预测概率
    auc = roc_auc_score(Y_test, Y_prob[:, 1])
    AUCS.append(auc)
    print(f'auc: {auc}')
print(f'Average AUC: {np.mean(AUCS)}')

NameError: name 'main' is not defined

main_df: Average AUC: 0.9204860054219773 <br>
processed:  Average AUC: 0.9200331315953896